#### Packages

In [34]:
import pandas as pd
import numpy as np
from liblinear.liblinearutil import *
import itertools
from scipy.sparse import lil_matrix
import math

#### Read Data

In [50]:
df = pd.read_csv("ML_HW4_data_train.dat", sep = "\t", header = None)
df.insert(loc=0, column="x0", value=1)
df.columns =['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'sign']
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()
print(X.shape)

df_test = pd.read_csv("ML_HW4_data_test.dat", sep = "\t", header = None)
df_test.insert(loc=0, column="x0", value=1)
X_t = df_test.iloc[:, :-1].to_numpy()
y_test = df_test.iloc[:, -1].to_numpy()

(200, 11)


##### Polynomial Transformation

In [36]:
def polynomial_features(X, degree):
    n_samples, n_features = X.shape
    combinations = list(itertools.combinations(range(n_features + degree - 1), degree))
    n_output_features = len(combinations)
    X_poly = np.empty((n_samples, n_output_features))
    for i, comb in enumerate(combinations):
        indices = np.array(list(comb)) - np.arange(degree)
        X_poly[:, i] = np.prod(X[:, indices], axis=1)
    return X_poly

X_train = polynomial_features(X, 4)
X_test = polynomial_features(X_t, 4)

#### 12

In [37]:
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)
    
# Evaluate the out-of-sample error for each value of lambda
Eout = []
for i in range(len(lambdas)):
    param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
    model = train(y, X_train, param)
    p_label, p_acc, p_vals = predict(y_test, X_test, model)
    predictions = [1 if pred > 0 else -1 for pred in p_label]
    Eout.append(np.mean(predictions != y_test))

# Select the best value of lambda
best_lambda = lambdas[np.argmin(Eout)]
print(math.log(best_lambda, 10))

[1e-06, 0.001, 1, 1000, 1000000]
Accuracy = 77.4% (387/500) (classification)
Accuracy = 82.2% (411/500) (classification)
Accuracy = 84.6% (423/500) (classification)
Accuracy = 85.8% (429/500) (classification)
Accuracy = 81.2% (406/500) (classification)
2.9999999999999996


#### 18

In [38]:
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)
    
# Evaluate the out-of-sample error for each value of lambda
Eout = []
for i in range(len(lambdas)):
    param = f'-s 6 -c {1/(lambdas[i])} -e 0.000001 -q'
    model = train(y, X_train, param)
    p_label, p_acc, p_valS = predict(y_test, X_test, model)
    predictions = [1 if pred > 0 else -1 for pred in p_label]
    Eout.append(np.mean(predictions != y_test))

# Select the best value of lambda
best_lambda = lambdas[np.argmin(Eout)]
print(math.log(best_lambda, 10))

[1e-06, 0.001, 1, 1000, 1000000]
Accuracy = 75.8% (379/500) (classification)
Accuracy = 83.8% (419/500) (classification)
Accuracy = 84.6% (423/500) (classification)
Accuracy = 68% (340/500) (classification)
Accuracy = 49.2% (246/500) (classification)
0.0


#### 13

In [39]:
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)
    
# Evaluate the out-of-sample error for each value of lambda
Ein = []
for i in range(len(lambdas)):
    param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
    model = train(y, X_train, param)
    p_label, p_acc, p_valS = predict(y, X_train, model)
    predictions = [1 if pred > 0 else -1 for pred in p_label]
    Ein.append(np.mean(predictions != y))

# Select the best value of lambda
best_lambda = lambdas[np.argmin(Eout)]
print(math.log(best_lambda, 10))

[1e-06, 0.001, 1, 1000, 1000000]
Accuracy = 100% (200/200) (classification)
Accuracy = 100% (200/200) (classification)
Accuracy = 100% (200/200) (classification)
Accuracy = 96% (192/200) (classification)
Accuracy = 76% (152/200) (classification)
0.0


#### 19

In [40]:
sparsity = 0
param = f'-s 6 -c 0.5 -e 0.000001 -q'
model = train(y, X_train, param)
w = np.array(model.get_decfun()[0])
sparsity = np.sum(abs(w)<= 10**(-6))
print(sparsity)

955


#### 20

In [41]:
sparsity = 0
param = f'-s 0 -c 0.0005 -e 0.000001 -q'
model = train(y, X_train, param)
w = np.array(model.get_decfun()[0])
sparsity = np.sum(abs(w)<= 10**(-6))
print(sparsity)

1


#### 14, 15

In [32]:
rng = np.random.default_rng()
def train_test_split(X, y, test_size=0.4, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    n_samples = X.shape[0]
    n_test = int(test_size * n_samples)
    test_indices = rng.choice(n_samples, n_test, replace=False)
    train_indices = np.delete(np.arange(n_samples), test_indices)
    
    X_train = X[train_indices]
    X_val = X[test_indices]
    y_train = y[train_indices]
    y_val = y[test_indices]
    
    return X_train, X_val, y_train, y_val
l=[-6,-3,0,3,6]
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)

Eout = []
count_best_lambda = [0] * len(lambdas)
num_iter = 256
for t in range(num_iter):
    b_acc = 0
    bl = -1
    Eval = []
    Xv_train, X_val, yv_train, y_val = train_test_split(X_train, y)
    for i in range(len(lambdas)):
        param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
        model = train(yv_train, Xv_train, param) 
        p_label, p_acc, p_vals = predict(y_val, X_val, model, '-q')
        #print(b_acc,p_acc[0])
        if b_acc<=p_acc[0]:
            b_acc=p_acc[0]
            bl = l[i]
            #print(bl)
            
        predictions = [1 if pred > 0 else -1 for pred in p_label]
        Eval.append(np.mean(predictions != y_val))
    # Select the best value of lambda
    #best_lambda_idx = np.argmin(Eval)
    #count_best_lambda[best_lambda_idx] += 1
    count_best_lambda[l.index(bl)] += 1
    best_param = f'-s 0 -c {1/(2*lambdas[l.index(bl)])} -e 0.000001 -q'
    #best_param = f'-s 0 -c {1/(2*lambdas[best_lambda_idx])} -e 0.000001 -q'
    new_model = train(yv_train, Xv_train, best_param) 
    pn_label, pn_acc, pn_vals = predict(y_test, X_test, new_model,'-q')

    Eout.append(1-pn_acc[0]/100)

print(count_best_lambda)
best_lambda_idx = np.argmax(count_best_lambda)
best_lambda = lambdas[best_lambda_idx]
print("Lambda that is selected the most often:", best_lambda)


[1e-06, 0.001, 1, 1000, 1000000]
[2, 16, 82, 153, 3]
Lambda that is selected the most often: 1000


In [33]:
print(np.mean(Eout))

0.1687734375


##### 15

In [57]:
Eout = []
num_iter = 256
for t in range(num_iter):
    Etemp = []
    Xv_train, X_val, yv_train, y_val = train_test_split(X_train, y)
    for i in range(len(lambdas)):
        param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
        model = train(yv_train, Xv_train, param) 
        p_label, p_acc, p_vals = predict(y_val, X_val, model, '-q')
        predictions = [1 if pred > 0 else -1 for pred in p_label]
        Etemp.append(np.mean(predictions != y_test))
    # Select the best value of lambda
    best_lambda_idx = np.argmin(Etemp)
    Eout.append(np.min(Etemp))
    
print(np.mean(Eout))   


<ipython-input-57-b30ce3367e1a>:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  Etemp.append(np.mean(predictions != y_test))


1.0


In [55]:
Eout = []
num_iter = 256
for t in range(num_iter):
    param = f'-s 0 -c {1/(2*1000)} -e 0.000001 -q'
    model = train(yv_train, Xv_train, param) 
    p_label, p_acc, p_vals = predict(y_test, X_test, model, '-q')
    predictions = [1 if pred > 0 else -1 for pred in p_label]
    Eout.append(np.mean(predictions != y_test))

print(np.mean(Eout))

0.18999999999999997


#### 16

In [31]:
l=[-6,-3,0,3,6]
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)

Eout = []
count_best_lambda = [0] * len(lambdas)
num_iter = 256
for t in range(num_iter):
    b_acc = 0
    bl = -1
    Eval = []
    Xv_train, X_val, yv_train, y_val = train_test_split(X_train, y)
    for i in range(len(lambdas)):
        param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
        model = train(yv_train, Xv_train, param) 
        p_label, p_acc, p_vals = predict(y_val, X_val, model, '-q')
        #print(b_acc,p_acc[0])
        if b_acc<=p_acc[0]:
            b_acc=p_acc[0]
            bl = l[i]
            #print(bl)
            
        predictions = [1 if pred > 0 else -1 for pred in p_label]
        Eval.append(np.mean(predictions != y_val))
    # Select the best value of lambda
    #best_lambda_idx = np.argmin(Eval)
    #count_best_lambda[best_lambda_idx] += 1
    count_best_lambda[l.index(bl)] += 1
    best_param = f'-s 0 -c {1/(2*lambdas[l.index(bl)])} -e 0.000001 -q'
    #best_param = f'-s 0 -c {1/(2*lambdas[best_lambda_idx])} -e 0.000001 -q'
    new_model = train(y, X_train, best_param) 
    pn_label, pn_acc, pn_vals = predict(y_test, X_test, new_model,'-q')
    #new_predictions = [1 if pred > 0 else -1 for pred in pn_label]
    #print(pn_acc)
    Eout.append(1-pn_acc[0]/100)

print(count_best_lambda)
best_lambda_idx = np.argmax(count_best_lambda)
best_lambda = lambdas[best_lambda_idx]
print("Lambda that is selected the most often:", best_lambda)
print("Eout = ", np.mean(Eout))


[1e-06, 0.001, 1, 1000, 1000000]
[2, 13, 77, 162, 2]
Lambda that is selected the most often: 1000
Eout =  0.14845312499999996


#### 17

In [49]:
n_folds = 5

l=[-6,-3,0,3,6]
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]
print(lambdas)

avgEcv = []
count_best_lambda = [0] * len(lambdas)
num_iter = 256
for t in range(num_iter):
    b_acc = 0
    bl = -1
    Ecv = []

    # Generate a random permutation of the indices
    indices = rng.permutation(X_train.shape[0])
    # Split the indices into n_folds equal parts
    fold_indices = np.array_split(indices, n_folds)

    # Loop over the folds
    for i in range(n_folds):
        # Split the data into training and validation sets
        val_indices = fold_indices[i]
        train_indices = np.concatenate([fold_indices[j] for j in range(n_folds) if j != i])
        Xv_train, yv_train = X_train[train_indices, :-1], y[train_indices]
        X_val, y_val = X_train[val_indices, :-1], y[val_indices]
    
    for i in range(len(lambdas)):
        param = f'-s 0 -c {1/(2*lambdas[i])} -e 0.000001 -q'
        model = train(yv_train, Xv_train, param) 
        p_label, p_acc, p_vals = predict(y_val, X_val, model, '-q')
        if b_acc<=p_acc[0]:
            b_acc=p_acc[0]
            bl = l[i]
            
        predictions = [1 if pred > 0 else -1 for pred in p_label]
        Ecv.append(np.mean(predictions != y_val))
    
    # Select the best value of lambda
    count_best_lambda[l.index(bl)] += 1
    best_param = f'-s 0 -c {1/(2*lambdas[l.index(bl)])} -e 0.000001 -q'
    new_model = train(yv_train, Xv_train, best_param) 
    pn_label, pn_acc, pn_vals = predict(y_val, X_val, new_model,'-q')
    avgEcv.append(1-pn_acc[0]/100)

print(count_best_lambda)
best_lambda_idx = np.argmax(count_best_lambda)
best_lambda = lambdas[best_lambda_idx]
print("Lambda that is selected the most often:", bl)
print("average Ecv = ", np.mean(avgEcv))



[1e-06, 0.001, 1, 1000, 1000000]
[4, 6, 70, 171, 5]
Lambda that is selected the most often: 3
average Ecv =  0.11464843749999999
